<a href="https://colab.research.google.com/github/aditya-007/Load-forecasting/blob/main/LSTM_load_forecast_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.optimizers import Adam
from sklearn.metrics import mean_squared_error


In [3]:
data = pd.read_csv('ercot-dataset.csv')

# Drop the 'Date' column since it is not needed for modeling
data_cleaned = data.drop(['Date'], axis=1)

# Display the first few rows and the column names to understand the dataset
data.head(), data.columns

(              Date  DryBulb  DewPnt  WetBulb  Humidity  ElecPrice  Day  Month  \
 0  01/01/2006 0:30     23.9   21.65    22.40      87.5      19.67    1      1   
 1  01/01/2006 1:00     23.9   21.70    22.40      88.0      18.56    1      1   
 2  01/01/2006 1:30     23.8   21.65    22.35      88.0      19.09    1      1   
 3  01/01/2006 2:00     23.7   21.60    22.30      88.0      17.40    1      1   
 4  01/01/2006 2:30     23.7   21.60    22.30      88.0      17.00    1      1   
 
    Year  Minutes     SYSLoad  
 0  2006       30  8013.27833  
 1  2006       60  7726.89167  
 2  2006       90  7372.85833  
 3  2006      120  7071.83333  
 4  2006      150  6865.44000  ,
 Index(['Date', 'DryBulb', 'DewPnt', 'WetBulb', 'Humidity', 'ElecPrice', 'Day',
        'Month', 'Year', 'Minutes', 'SYSLoad'],
       dtype='object'))

In [4]:
# Normalize the dataset using MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(data_cleaned)

# Define the target variable (SYSLoad) and features (everything else)
target_index = data.columns.get_loc('SYSLoad') - 1 # Adjusted due to dropped Date column

In [5]:
# Create sequences for LSTM model input
def create_sequences_lstm(data, target_index, time_step):
    X, Y = [], []
    for i in range(len(data) - time_step):
        X.append(data[i:i+time_step, :])  # All features over the time step
        Y.append(data[i+time_step, target_index])  # Target value at the next time step
    return np.array(X), np.array(Y)

# Define a time step (how many previous data points to consider)
time_step = 60

# Create the sequences
X, Y = create_sequences_lstm(data_scaled, target_index, time_step)


In [6]:
# Split data into training and testing sets (80% train, 20% test)
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
Y_train, Y_test = Y[:train_size], Y[train_size:]

# Check the shapes of the prepared training and test sets
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((70070, 60, 10), (17518, 60, 10), (70070,), (17518,))

In [7]:
# Function to build LSTM1, LSTM2, or LSTM3
def build_lstm_model(model_type, time_step, input_dim):
    model = Sequential()

    if model_type == 'LSTM1':
        # LSTM1 (Simple Model): One LSTM layer with 50 units
        model.add(LSTM(units=50, return_sequences=False, input_shape=(time_step, input_dim)))

    elif model_type == 'LSTM2':
        # LSTM2 (My Complex Model): Two LSTM layers with 100 and 50 units
        model.add(LSTM(units=100, return_sequences=True, input_shape=(time_step, input_dim)))
        model.add(LSTM(units=50, return_sequences=False))

    elif model_type == 'LSTM3':
        # LSTM3 (Proposed Complex Model): Two LSTM layers with Dropout and ReLU activation
        model.add(LSTM(units=128, return_sequences=True, input_shape=(time_step, input_dim)))
        model.add(Dropout(0.2))  # Dropout after the first LSTM layer
        model.add(LSTM(units=64, return_sequences=False))
        model.add(Dropout(0.2))  # Dropout after the second LSTM layer
        model.add(Dense(units=64, activation='relu'))  # Dense layer with ReLU activation

    # Output layer for SYSLoad prediction
    model.add(Dense(units=1))

    # Compile the model
    model.compile(optimizer=Adam(), loss='mean_squared_error')

    return model

# Select model type ('LSTM1', 'LSTM2', 'LSTM3')
model_type = 'LSTM3'  # You can change this to 'LSTM1' or 'LSTM2'

# Build the selected model
input_dim = X_train.shape[2]  # Number of features (input dimensions)
model = build_lstm_model(model_type, time_step, input_dim)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
# Train the model on the training set
history = model.fit(X_train, Y_train, epochs=10, batch_size=64, validation_data=(X_test, Y_test), verbose=1)

Epoch 1/10
1095/1095 ━━━━━━━━━━━━━━━━━━━━ 227s 207ms/step - loss: 0.0024 - val_loss: 6.8193e-04
Epoch 2/10
1095/1095 ━━━━━━━━━━━━━━━━━━━━ 216s 197ms/step - loss: 9.8342e-04 - val_loss: 4.0501e-04
Epoch 3/10
1095/1095 ━━━━━━━━━━━━━━━━━━━━ 264s 199ms/step - loss: 5.6683e-04 - val_loss: 2.1040e-04
Epoch 4/10
1095/1095 ━━━━━━━━━━━━━━━━━━━━ 262s 199ms/step - loss: 4.3357e-04 - val_loss: 3.2684e-04
Epoch 5/10
1095/1095 ━━━━━━━━━━━━━━━━━━━━ 259s 197ms/step - loss: 3.6891e-04 - val_loss: 3.5423e-04
Epoch 6/10
 880/1095 ━━━━━━━━━━━━━━━━━━━━ 39s 184ms/step - loss: 3.0859e-04

In [ ]:
# Make predictions on the training and testing sets
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

In [ ]:
# Inverse transform the predictions and the actual values to get the original scale
train_predict = scaler.inverse_transform(
    np.concatenate((np.zeros((train_predict.shape[0], data_cleaned.shape[1] - 1)), train_predict), axis=1)
)[:, -1]  # We add back the other features with zeros before inverse transforming

test_predict = scaler.inverse_transform(
    np.concatenate((np.zeros((test_predict.shape[0], data_cleaned.shape[1] - 1)), test_predict), axis=1)
)[:, -1]

# Similarly, inverse transform the actual SYSLoad values for Y_train and Y_test
Y_train_original = scaler.inverse_transform(
    np.concatenate((np.zeros((Y_train.shape[0], data_cleaned.shape[1] - 1)), Y_train.reshape(-1, 1)), axis=1)
)[:, -1]

Y_test_original = scaler.inverse_transform(
    np.concatenate((np.zeros((Y_test.shape[0], data_cleaned.shape[1] - 1)), Y_test.reshape(-1, 1)), axis=1)
)[:, -1]

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# Calculate the Mean Absolute Percentage Error (MAPE) for training and testing sets
test_mape = mean_absolute_percentage_error(Y_test_original, test_predict)

test_mse = mean_squared_error(Y_test_original, test_predict)

print(f"Test MAPE: {test_mape:.2f}%")
print(f"Test MSE: {test_mse:.2f}")

In [ ]:
# Plot the first 24 actual vs predicted values for the training set
plt.figure(figsize=(14, 6))
plt.plot(Y_train_original[:48], label='Actual SYSLoad (Train)', color="blue")
plt.plot(train_predict[:48], label='Predicted SYSLoad (Train)', color="red")
plt.title('Training Set: Actual vs Predicted SYSLoad (First 24 Points)')
plt.xlabel('Time Steps')
plt.ylabel('SYSLoad')
plt.legend()
plt.show()

# Plot the first 24 actual vs predicted values for the testing set
plt.figure(figsize=(14, 6))
plt.plot(Y_test_original[:48], label='Actual SYSLoad (Test)', color="blue")
plt.plot(test_predict[:48], label='Predicted SYSLoad (Test)', color="red")
plt.title('Testing Set: Actual vs Predicted SYSLoad (First 24 Points)')
plt.xlabel('Time Steps')
plt.ylabel('SYSLoad')
plt.legend()
plt.show()